# Data Prep1 : start from raw data to cleaned and created binary(y)

# Import raw data

In [1]:
import pandas as pd
import re
import os
import codecs,pickle
a={}
path = 'orchid97.crp.utf'
lines=codecs.open(path, 'r',encoding='utf8')
text = lines.read()

In [2]:
lines = text.split('\n')

In [3]:
len(lines)

431338

In [4]:
lines

['%TTitle: การประชุมทางวิชาการ ครั้งที่ 1',
 '%ETitle: [1st Annual Conference]',
 '%TAuthor:',
 '%EAuthor:',
 '%TInbook: การประชุมทางวิชาการ ครั้งที่ 1, โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์, ปีงบประมาณ 2531, เล่ม 1',
 '%EInbook: The 1st Annual Conference, Electronics and Computer Research and Development Project, Fiscal Year 1988, Book 1',
 '%TPublisher: ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ, กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน',
 '%EPublisher: National Electronics and Computer Technology Center, Ministry of Science, Technology and Energy',
 '%Page:',
 '%Year: 1989',
 '%File:',
 '#P1',
 '#1',
 'การประชุมทางวิชาการ ครั้งที่ 1//',
 'การ/FIXN',
 'ประชุม/VACT',
 'ทาง/NCMN',
 'วิชาการ/NCMN',
 '<space>/PUNC',
 'ครั้ง/CFQC',
 'ที่ 1/DONM',
 '//',
 '#2',
 'โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์//',
 'โครงการวิจัยและพัฒนา/NCMN',
 'อิเล็กทรอนิกส์/NCMN',
 'และ/JCRG',
 'คอมพิวเตอร์/NCMN',
 '//',
 '#3',
 'ปีงบประมาณ 2531//',
 'ปีงบประมาณ/NCMN',
 '<space>/PUN

# Remove junk lines

In [5]:
lines = [ x for x in lines if "%" not in x and "#P" not in x]

In [6]:
lines

['#1',
 'การประชุมทางวิชาการ ครั้งที่ 1//',
 'การ/FIXN',
 'ประชุม/VACT',
 'ทาง/NCMN',
 'วิชาการ/NCMN',
 '<space>/PUNC',
 'ครั้ง/CFQC',
 'ที่ 1/DONM',
 '//',
 '#2',
 'โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์//',
 'โครงการวิจัยและพัฒนา/NCMN',
 'อิเล็กทรอนิกส์/NCMN',
 'และ/JCRG',
 'คอมพิวเตอร์/NCMN',
 '//',
 '#3',
 'ปีงบประมาณ 2531//',
 'ปีงบประมาณ/NCMN',
 '<space>/PUNC',
 '2531/NCNM',
 '//',
 '#4',
 'เล่ม 1//',
 'เล่ม/CNIT',
 '<space>/PUNC',
 '1/DONM',
 '//',
 '#1',
 'ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ//',
 'ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ/NPRP',
 '//',
 '#2',
 'กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน//',
 'กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน/NPRP',
 '//',
 '#1',
 'วันที่ 15-16 สิงหาคม 2532//',
 'วัน/NCMN',
 'ที่ 15/DONM',
 '<minus>/PUNC',
 '16/DONM',
 '<space>/PUNC',
 'สิงหาคม/NCMN',
 '<space>/PUNC',
 '2532/NCNM',
 '//',
 '#2',
 'ห้องประชุม ชั้น 4//',
 'ห้องประชุม/NCMN',
 '<space>/PUNC',
 'ชั้น/CNIT',
 '<space>/PUNC',
 '4/NCNM',
 '//',


# Extract sentences and their words

In [7]:
lines.count("//")

23125

In [8]:
from itertools import  groupby

In [37]:
#creat list of sentences
result = []
start = True
for element in lines:
    if start and element!="//":  # start group
        result.append([element])
        start = False
    elif element!="//":  # just append to current group
        result[-1].append(element)
    else:  # close group
        start = True


In [38]:
#creat dicts of sentences and their words
result_dict = dict()
for i in range(len(result)):
    words = []
    for j in range(len(result[i][1:])):
        if result[i][1:][j][-2:]=='//':
            words.append(result[i][1:][j])
            sentences = ' '.join(map(str, words))
            sentences = sentences.replace('\\\\', '')
            sentences = sentences.replace('//', '')
        words.append(result[i][1:][j])
    words = [ x for x in words if "\\\\" not in x and "//" not in x]
    result_dict[sentences] = words

In [39]:
#remove NERs keep only words
new_dict = dict()
for i in result_dict:
    new_words = []
    for word in result_dict[i]:
        new_words.append(word.split("/")[0])
        new_words = [ x for x in new_words if "<space>" not in x]
    new_dict[i] = new_words

In [42]:
#creat dict to dataframe of sentences and their words
dataset = pd.DataFrame(list(new_dict.items()),columns = ['sentence(x)','words']) 
pd.set_option('display.max_colwidth', -1)

In [43]:
dataset.head(10)

,sentence(x),words
0,การประชุมทางวิชาการ ครั้งที่ 1,"[การ, ประชุม, ทาง, วิชาการ, ครั้ง, ที่ 1]"
1,โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์,"[โครงการวิจัย, และ, พัฒนา, อิเล็กทรอนิกส์, และ, คอมพิวเตอร์]"
2,ปีงบประมาณ 2531,"[ปีงบประมาณ, 2531]"
3,เล่ม 1,"[เล่ม, 1]"
4,ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ,[ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ]
5,กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน,[กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน]
6,วันที่ 15-16 สิงหาคม 2532,"[วัน, ที่ 15, <minus>, 16, สิงหาคม, 2532]"
7,ห้องประชุม ชั้น 4,"[ห้องประชุม, ชั้น, 4]"
8,สาร,[สาร]
9,ฯพณฯ รัฐมนตรีว่าการกระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน,"[ฯพณฯ, รัฐมนตรีว่าการ, กระทรวงวิทยาศาสตร์ เทคโนโลยีและการพลังงาน]"


# Return special chars

In [15]:
special_char = pd.read_excel('special_char.xlsx')

In [16]:
special_char

,Defined_strings,Special_characters
0,<space>,
1,<exclamation>,!
2,<quotation>,""""
3,<number>,#
4,<slash>,/
5,<colon>,:
6,<semi_colon>,;
7,<less_than>,<
8,<dollar>,$
9,<percent>,%


In [17]:
dict_special_char = dict([(a,b) for a,b in zip(special_char.Defined_strings,special_char.Special_characters)])

In [19]:
def check_special(lst):
    special = []
    for i in lst:
        
        if i in dict_special_char:
            special.append(dict_special_char[i])
        else:
            i = i.replace(" ", "")
            special.append(i)
    return(special)

In [20]:
#dataset.to_csv("dataset_orchid.csv")

In [21]:
dataset['word_with_special_char(y)'] = dataset['words'].apply(check_special)

In [22]:
dataset.tail(20)

,sentence(x),words,word_with_special_char(y)
20688,"หมายเหตุ ตารางที่ 1, 2 เชื่อมโยงด้วย HN","[หมายเหตุ, ตาราง, ที่ 1, <comma>, 2, เชื่อมโยง, ด้วย, HN]","[หมายเหตุ, ตาราง, ที่1, ,, 2, เชื่อมโยง, ด้วย, HN]"
20689,#1 การจำแนกการทำงานของกล้ามเนื้อลิ้นขณะเคี้ยวอาหารและขณะกลืนโดยใช้ surface electromyograms,"[#1, การ, จำแนก, การ, ทำงาน, ของ, กล้ามเนื้อ, ลิ้น, ขณะ, เคี้ยว, อาหาร, และ, ขณะ, กลืน, โดย, ใช้, surface electromyograms]","[#1, การ, จำแนก, การ, ทำงาน, ของ, กล้ามเนื้อ, ลิ้น, ขณะ, เคี้ยว, อาหาร, และ, ขณะ, กลืน, โดย, ใช้, surfaceelectromyograms]"
20690,"#1 วิทูร ลีลามานิตย์,1 ชูศักดิ์ ลิ่มสกุล,2 อลัน กีเตอร์,3","[#1, วิทูร, ลีลามานิตย์, <comma>, 1, ชูศักดิ์, ลิ่มสกุล, <comma>, 2, อลัน, กีเตอร์, <comma>, 3]","[#1, วิทูร, ลีลามานิตย์, ,, 1, ชูศักดิ์, ลิ่มสกุล, ,, 2, อลัน, กีเตอร์, ,, 3]"
20691,1 ภาควิชาโสต นาสิก ลาริงซ์วิทยา คณะแพทยศาสตร์,"[1, ภาควิชา, โสต, นาสิก, ลาริงซ์วิทยา, คณะ, แพทยศาสตร์]","[1, ภาควิชา, โสต, นาสิก, ลาริงซ์วิทยา, คณะ, แพทยศาสตร์]"
20692,2 ภาควิชาไฟฟ้า คณะวิศวกรรมศาสตร์,"[2, ภาควิชา, ไฟฟ้า, คณะ, วิศวกรรมศาสตร์]","[2, ภาควิชา, ไฟฟ้า, คณะ, วิศวกรรมศาสตร์]"
20693,3 หน่วยระบาดวิทยา คณะแพทยศาสตร์,"[3, หน่วยระบาดวิทยา, คณะ, แพทยศาสตร์]","[3, หน่วยระบาดวิทยา, คณะ, แพทยศาสตร์]"
20694,มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ สงขลา 90112,"[มหาวิทยาลัยสงขลานครินทร์, วิทยาเขต, หาดใหญ่, สงขลา, 90112]","[มหาวิทยาลัยสงขลานครินทร์, วิทยาเขต, หาดใหญ่, สงขลา, 90112]"
20695,เพื่อจำแนกการทำงานของกล้ามเนื้อลิ้นขณะกลืนอาหารจากการทำงานอื่น ซึ่งเกี่ยวข้องกับการรับประทานอาหาร,"[เพื่อ, จำแนก, การ, ทำงาน, ของ, กล้ามเนื้อ, ลิ้น, ขณะ, กลืน, อาหาร, จาก, การ, ทำงาน, อื่น, ซึ่ง, เกี่ยวข้อง, กับ, การ, รับประทาน, อาหาร]","[เพื่อ, จำแนก, การ, ทำงาน, ของ, กล้ามเนื้อ, ลิ้น, ขณะ, กลืน, อาหาร, จาก, การ, ทำงาน, อื่น, ซึ่ง, เกี่ยวข้อง, กับ, การ, รับประทาน, อาหาร]"
20696,คณะผู้ประพันธ์จึงได้ทำการบันทึกการทำงานของกล้ามเนื้อโคนลิ้นขณะเคี้ยวและกลืนอาหารอย่างละ 3 ครั้ง ในอาสาสมัคร 14 คนที่ไม่มีอาการผิดปรกติ,"[คณะ, ผู้ประพันธ์, จึง, ได้, ทำ, การ, บันทึก, การ, ทำงาน, ของ, กล้ามเนื้อ, โคน, ลิ้น, ขณะ, เคี้ยว, และ, กลืน, อาหาร, อย่าง, ละ, 3, ครั้ง, ใน, อาสาสมัคร, 14, คน, ที่, ไม่, มี, อาการ, ผิดปรกติ]","[คณะ, ผู้ประพันธ์, จึง, ได้, ทำ, การ, บันทึก, การ, ทำงาน, ของ, กล้ามเนื้อ, โคน, ลิ้น, ขณะ, เคี้ยว, และ, กลืน, อาหาร, อย่าง, ละ, 3, ครั้ง, ใน, อาสาสมัคร, 14, คน, ที่, ไม่, มี, อาการ, ผิดปรกติ]"
20697,โดยวิธี surface electromyography (SEMG) Waveform ที่ได้จาก SEMG,"[โดย, วิธี, surface electromyography, <left_parenthesis>, SEMG, <right_parenthesis>, Waveform, ที่, ได้, จาก, SEMG]","[โดย, วิธี, surfaceelectromyography, (, SEMG, ), Waveform, ที่, ได้, จาก, SEMG]"


# Create Binary sequence of answers and clean whitespace

In [23]:
list_special_char = []
for i in dict_special_char:
    list_special_char.append(dict_special_char[i])

In [24]:
def to_binary(lst):
    binary_str = ""
    for i in lst:
        i = str(i).replace(" ", "")
        binary = '1'
        if i in list_special_char:
            binary = binary
        elif str(i).isnumeric():
            for j in range(len(str(i))-1):
                binary = binary+'0'            
        else:
            for j in range(len(i)-1):
                binary = binary+'0'
        binary_str = binary_str+binary
    return(binary_str)

In [25]:
def check_char(lst):
    result = []
    for i in lst:
        if str(i).isnumeric() or i in list_special_char:
            result.append(False)
        else:
            result.append(True)
    return(result)

In [26]:
#dataset['check_char'] = dataset['word_with_special_char'].apply(check_char)

In [27]:
dataset['sentence_without_space(x_clean)'] = dataset['sentence(x)'].replace(" ", "")

In [28]:
dataset['binary(y)'] = dataset['word_with_special_char(y)'].apply(to_binary)

# Clean records that len(sentence)!=len(binary answer)

In [29]:
def str_len(lst):
    lst = lst.replace(" ", "")
    return(len(lst))

In [30]:
dataset['sentence_len'] = dataset['sentence_without_space(x_clean)'].apply(str_len)

In [31]:
dataset['binary_len'] = dataset['binary(y)'].apply(str_len)

In [32]:
dataset_clean = dataset[dataset['sentence_len']==dataset['binary_len']]

In [33]:
#dataset_clean.to_csv("dataset_orchid_clean.csv")

In [34]:
dataset.shape

(20708, 7)

In [35]:
dataset_clean.shape

(20084, 7)

In [36]:
dataset_clean.head()

,sentence(x),words,word_with_special_char(y),sentence_without_space(x_clean),binary(y),sentence_len,binary_len
0,การประชุมทางวิชาการ ครั้งที่ 1,"[การ, ประชุม, ทาง, วิชาการ, ครั้ง, ที่ 1]","[การ, ประชุม, ทาง, วิชาการ, ครั้ง, ที่1]",การประชุมทางวิชาการ ครั้งที่ 1,1001000001001000000100001000,28,28
1,โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์,"[โครงการวิจัย, และ, พัฒนา, อิเล็กทรอนิกส์, และ, คอมพิวเตอร์]","[โครงการวิจัย, และ, พัฒนา, อิเล็กทรอนิกส์, และ, คอมพิวเตอร์]",โครงการวิจัยและพัฒนาอิเล็กทรอนิกส์และคอมพิวเตอร์,100000000000100100001000000000000010010000000000,48,48
2,ปีงบประมาณ 2531,"[ปีงบประมาณ, 2531]","[ปีงบประมาณ, 2531]",ปีงบประมาณ 2531,10000000001000,14,14
3,เล่ม 1,"[เล่ม, 1]","[เล่ม, 1]",เล่ม 1,10001,5,5
4,ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ,[ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ],[ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ],ศูนย์เทคโนโลยีอิเล็กทรอนิกส์และคอมพิวเตอร์แห่งชาติ,10000000000000000000000000000000000000000000000000,50,50
